In [1]:
import sys
sys.setrecursionlimit(1500)

In [2]:
def inputMaze(name="tinySearch.txt"):
    with open(name) as mazeTxt:
        maze = [list(line)[0:-1] for line in mazeTxt]
        rows = len(maze)
        maze[rows-1].append('%')
    return maze

def inputMaze1(name="tinySearch.txt"):
    with open(name) as mazeTxt:
        maze = [list(line)[:] for line in mazeTxt]
        rows = len(maze)
    return maze

In [ ]:
#maze = inputMaze()
#print(maze)
#print len(maze)
#for i in range(len(maze)):
#    print len(maze[i])

##### let's try to implement the new state representation and start with BFS

In [3]:
tiny_search = inputMaze("tinySearch.txt")
small_search = inputMaze1("smallSearch.txt")
medium_search = inputMaze1("mediumSearch.txt")

In [4]:
print(tiny_search)

cur_search = small_search
print(len(cur_search))
for i in range(len(cur_search)):
    print(len(cur_search[i]))


[['%', '%', '%', '%', '%', '%', '%', '%', '%', '%'], ['%', '.', ' ', ' ', '%', ' ', '.', ' ', ' ', '%'], ['%', ' ', '%', '.', '%', ' ', '%', '%', ' ', '%'], ['%', ' ', '%', ' ', ' ', ' ', '.', '%', '.', '%'], ['%', ' ', '.', '%', 'P', '%', ' ', ' ', ' ', '%'], ['%', '.', ' ', ' ', '.', ' ', ' ', '.', ' ', '%'], ['%', ' ', '%', '%', '%', '%', ' ', '%', ' ', '%'], ['%', '.', ' ', '.', ' ', ' ', ' ', '%', '.', '%'], ['%', '%', '%', '%', '%', '%', '%', '%', '%', '%']]
13
31
31
31
31
31
31
31
31
31
31
31
31
31


In [5]:
# this func returns 3 things: start coordinate, dot_positions and the number of dots
def findStartNDots(maze):
    dot_positions = set()   # dot_pos set will be used to hold dot coordinates in form of tuples
    dot_count = 0
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'P':
                start = (i,j)
            if maze[i][j] == '.':
                dot_count += 1
                dot_positions.add((i,j))
    return [start, dot_positions, dot_count]

#start, dot_pos, N = findStartNDots(tiny_search)
#print(start, dot_pos, N)

In [6]:
# must be called in wrapper! (dot_pos) should be available when calling this func
# check whether cur_pos is on dot
# input: cor is a (x, y) tuple
def isDot(cor, dot_pos):
    if cor in dot_pos:
        return True
    else:
        return False

In [7]:
def multi_valid_neighbors(cur_state, maze, height, width, dot_pos):
    y = cur_state[0][0]
    x = cur_state[0][1]
    cur_pos = (y, x)
    
    
    valid_neighbors = []
    if (x-1) >= 0 and maze[y][x-1] != '%':
        new_pos = (y, x-1)
        cur_eaten = set(cur_state[1])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
        else:                  
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
    if (y+1) < height and maze[y+1][x] != '%':
        new_pos = (y+1, x)
        cur_eaten = set(cur_state[1])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
        else:
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )                           
    if (x+1) < width and maze[y][x+1] != '%':
        new_pos = (y, x+1)
        cur_eaten = set(cur_state[1])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
        else:
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
                                   
    if (y-1) >= 0 and maze[y-1][x] != '%':
        new_pos = (y-1, x)
        cur_eaten = set(cur_state[1])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
        else:
            valid_neighbors.append( (new_pos, frozenset(cur_eaten)) )
                                   
    return valid_neighbors

In [20]:
#
# !! when using set inside a set, we have to use frozenset(), so that it is immutable,
# !! Actually tuple is just immutable
def multiBfs(search):
    height = len(search)
    width = len(search[0])
    
    
    start_pos, dot_pos, N = findStartNDots(search)
    # start state is in the form of ((x,y),{visited dots})
    start_state = (start_pos, frozenset())
    
    
    # visited is a set of state which has the same form as start_state
    # so, { state0, state1, ...}
    visited = set()
    # set up the dict for the track
    track = {}
    
    
    # use queue for BFS implementation
    # set up the queue
    # the queue holds states like in the same form as start_state
    queue = []
    queue.insert(0, start_state)
    # add start_state to visited
    visited.add(start_state)

    count = 0
    success = False
    final_state = start_state
    while queue:
        count += 1
        cur = queue.pop()
        if len(cur[1]) == N:       # BASE CASE: if we have visited/eaten all the dots
            success = True
            final_state = cur
            break
            
        neighbors = multi_valid_neighbors(cur, search, height, width, dot_pos)
        for nei in neighbors:
            if nei!= None:
                if nei not in visited:
                    queue.insert(0,nei)
                    visited.add(nei)
                    track[nei] = cur
                    
        if count%100000 == 0:
            print(count)
            
    path = []
    c = 0
    cur = final_state
    while len(cur[1]) != 0:
        c += 1
        path.insert(0, cur[0])
        cur = track[cur]
        if c%100000 == 0:
            print(c)
    return success, path


In [21]:
_, path = multiBfs(tiny_search)

In [23]:
print(path)
print(len(path))

[(5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (6, 8), (7, 8), (6, 8), (5, 8), (4, 8), (3, 8), (2, 8), (1, 8), (1, 7), (1, 6), (1, 5), (2, 5), (3, 5), (3, 6), (3, 5), (3, 4), (3, 3), (2, 3), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (4, 2), (4, 1), (5, 1), (6, 1), (7, 1), (7, 2), (7, 3)]
36


In [24]:
# 此cell为试验场，以下内容请忽略

## !!! a var in a loop is not valid outside of it
#for i in range(8):
#    cur += i
#print(cur)
   

#dict = {}
#print(type(dict))

#b = {1,2,3}
#def scope(b):
#    b = set()
#    print("inner", len(b))

#scope(b)    
#print("outer", b)

## no bad thing will happen if we add already existing element into set
#a = {1,2}
#a.add(2)
#print(a)

# local var in outer func cannot be automatically passed into inner func, but global var can
'''
def inner():
    print(a)
    
def outer():
    a = 10
    print("fuck")
    inner()
    print("fuck")
outer()
'''

'''
f = frozenset([1,2,3])
print("f type is ", type(f))

b = set(f)
print("b type is", type(b))
print(b)
b.add(9)
print(b)
'''
#ff = frozenset(b)
#print(type(ff))

NameError: name 'cur' is not defined